<a href="https://colab.research.google.com/github/imeldp96/qsar_study/blob/main/Support_Vector_Regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data preparation

In [ ]:
!pip install kennard_stone #to install kennard-stone splitting algorithm

In [ ]:
#to mount google drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
# to import dependencies

import pandas as pd
import numpy as np
from pandas import DataFrame as df
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score, r2_score, mean_absolute_percentage_error
from sklearn import svm
from sklearn.preprocessing import StandardScaler
#import kennard_stone as ks #activate this line to install ks as well (after installing), deactivate if not needed

random = 42 #random number for consistent results, can be changed to anything

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/data/all.csv', index_col=[0], header=[0]) #change accordingly with the location of your csv file

In [ ]:
#X= data.drop('pIC50', axis=1) #Select the descriptor's columns

#X=data[['X3', 'X21','X11']] #vi-qc
#X=data[['X11', 'X15', 'X16']] #ga-qc

y = data['pIC50']  # Setting y as the target variable.
X.head()

In [ ]:
#Kennard-Stone Algorithm for training-test set division
#X_train, X_test, y_train, y_test = ks.train_test_split(X, y, test_size=0.2) #change test_size accordingly with the proportion of training-test set

In [ ]:
#Dont run this if you run the KS set division
#only run this if the training and test set has been divided into different rows

X_train = X.iloc[:19] #select rows for train set
X_test = X.iloc[19:28] #select rows for test set

y_train = y.iloc[:19]
y_test = y.iloc[19:28]

In [ ]:
print(y_test)

In [ ]:
#standardization of X values
#svr is very sensitive to scale of the features/descriptors, thus standardization is needed
scaler = StandardScaler()

# Fit the scaler to your training data and transform it
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data using the same scaler
X_test_scaled = scaler.transform(X_test)

print(X_train_scaled)

#SVR Model Building

In [ ]:
# fitting
svr=svm.SVR()
svr_fit = svr.fit(X_train_scaled, y_train)

In [ ]:
#prediction on the training dataset
ytrain_pred = svr_fit.predict(X_train_scaled)
#training model evaluation

#training r-sq
print('The training r_sq is: %.3f'% svr_fit.score(X_train_scaled, y_train))

#RMSE
print('The RMSE is: %.3f'% np.sqrt(mean_squared_error(y_train, ytrain_pred)))

#MAPE
print('The MAPE is: %.3f'% mean_absolute_percentage_error(y_train, ytrain_pred))

In [ ]:
#prediction on the testing data
ytest_pred = svr_fit.predict(X_test_scaled)

#testing coef. of determination
print('The testing r_sq is: %.3f'% r2_score(y_test, ytest_pred))
#model evaluation metrics on test set

#RMSE
print('The RMSE is: %.3f'% np.sqrt(mean_squared_error(y_test, ytest_pred)))

#MAPE
print('The MAPE is: %.3f'% mean_absolute_percentage_error(y_test, ytest_pred))

In [ ]:
#get the predicted targets
df_ytrain = pd.DataFrame(y_train)
df_ytrainpred = pd.DataFrame(ytrain_pred)
df_ytest = pd.DataFrame(y_test)
df_ytestpred = pd.DataFrame(ytest_pred)

In [ ]:
#make pred vs exp graph

import numpy as np
import matplotlib.pyplot as plt

def plot_scatter_with_line(y_train, y_train_pred, y_test, y_test_pred):
    fig = plt.figure(figsize=(8, 8))  # Set the figure size
    plt.scatter(y_train, y_train_pred, color='blue', label='Training Set', alpha=0.5, s=70)
    plt.scatter(y_test, y_test_pred, color='red', label='Test Set', alpha=0.5, s=50)
    plt.plot([np.min(np.concatenate([y_train, y_test])), np.max(np.concatenate([y_train, y_test]))],
             [np.min(np.concatenate([y_train, y_test])), np.max(np.concatenate([y_train, y_test]))],
             color='black', linestyle='--')  # Diagonal line
    plt.title('Experimental vs Predicted LogKow', fontsize=16)
    plt.xlabel('Experimental LogKow', fontsize=14)
    plt.ylabel('Predicted LogKow', fontsize=14)
    plt.axis('square')  # Set aspect ratio to be equal
    plt.legend(fontsize=12)
    plt.grid(False)

    # Set face color of the figure to white
    fig.patch.set_facecolor('none')

    plt.show()

# Assuming you have y_train, ytrain_pred, y_test, and ytest_pred as arrays
plot_scatter_with_line(y_train, ytrain_pred, y_test, ytest_pred)

#Hyperparameters Tuning

In [ ]:
import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the new specifications
param_grid = [
    {
        'kernel': ['rbf', 'linear'],
        'epsilon': np.arange(0.1, 1.01, 0.01).tolist(),
        'C': np.arange(1, 501, 1).tolist()
    },
    {
        'kernel': ['poly'],
        'epsilon': np.arange(0.1, 1.01, 0.01).tolist(),
        'C': np.arange(1, 501, 1).tolist(),
        'degree': np.arange(1, 21, 1).tolist()
    }
]

# Create a base model
svr = svm.SVR()

# Instantiate the grid search model
grid_search = GridSearchCV(
    estimator=svr,
    param_grid=param_grid,
    cv=8,
    n_jobs=-1,
    verbose=2,
    scoring=['r2', 'neg_root_mean_squared_error'],
    refit='neg_root_mean_squared_error',
    error_score='raise'
)


In [ ]:
# Fit the grid search to the data
grid_search.fit(X_train_scaled, y_train)
grid_search.best_params_

In [ ]:
# Access the randomized hyperparameters and their values
grids_params = grid_search.cv_results_

# Convert the randomized hyperparameters to a DataFrame
hyperparameters_df = pd.DataFrame(grids_params)

# Save the DataFrame to a CSV file
hyperparameters_df.to_csv('grids_hyperparameters.csv')